# Benchmarks

## Initialize

In [1]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
!pip install nancorrmp

In [3]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_medical_records"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

experiment = 220413
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [4]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").sort_values("endpoint")

In [4]:
#endpoints = [e[:-6] for e in data_outcomes.columns if "_event" in e]
endpoints = [
    "phecode_008",
    "phecode_092-2",
    "phecode_105",
    "phecode_107-2",
    "phecode_164",
    "phecode_202-2",
    "phecode_284",
    "phecode_292",
    "phecode_324-11",
    "phecode_328",
    "phecode_371",
    "phecode_401",
    "phecode_404",
    "phecode_424",
    "phecode_440-11",
    "phecode_468",
    "phecode_474",
    "phecode_522-1",
    "phecode_542-1",
    "phecode_581-1",
    "phecode_583",
    "phecode_665",
    "phecode_705-1",
    "OMOP_4306655"  
]

In [5]:
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_220412.feather").set_index("eid")
data_outcomes = data_outcomes[[c for c in data_outcomes.columns if "_event" in c and c[:-6] in endpoints]]

In [5]:
data_shap = pd.read_feather(f"{experiment_path}/shap_local.feather")
records = [r for r in data_shap.record if 
           "age_at_recruitment_f21022_0_0" not in r and
           "sex_f31_0_0_Female" not in r and
           "sex_f31_0_0_Male" not in r and
           "phecode_" not in r]

In [6]:
len(records)

19366

In [7]:
data_records = pd.read_feather(f"{output_path}/baseline_records_220412.feather", columns=["eid"] + records).set_index("eid")

In [8]:
data_all = data_records.merge(data_outcomes, left_index=True, right_index=True, how="left")

In [9]:
data_all

,OMOP_1000560,OMOP_1000632,OMOP_1000772,OMOP_1000995,OMOP_1036157,OMOP_1036228,OMOP_1036252,OMOP_1036487,OMOP_1036525,OMOP_1036690,...,phecode_424_event,phecode_328_event,phecode_468_event,phecode_440-11_event,phecode_284_event,phecode_008_event,phecode_092-2_event,phecode_581-1_event,phecode_292_event,phecode_542-1_event
eid,,,,,,,,,,,,,,,,,,,,,
1000018,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000020,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000043,False,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1000051,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025150,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6025165,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6025173,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
eligable_eids = pd.read_feather(f"{output_path}/eligable_eids_220414.feather")
eids_dict = eligable_eids.set_index("endpoint")["eid_list"].to_dict()

In [11]:
record_freqs = data_records.mean().sort_values(ascending=False)
record_freqs

OMOP_4081598     0.615561
OMOP_4052351     0.540684
OMOP_4061103     0.527156
OMOP_4144272     0.493337
OMOP_4057411     0.443337
                   ...   
OMOP_40482194    0.000050
OMOP_4113107     0.000050
OMOP_4122590     0.000050
OMOP_434951      0.000050
OMOP_4306655     0.000014
Length: 19366, dtype: float64

In [12]:
import ray

ray.init(num_cpus=24, include_dashboard=False)#dashboard_port=24763, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

{'node_ip_address': '10.32.105.13',
 'raylet_ip_address': '10.32.105.13',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-04-26_17-11-39_384698_1636001/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-04-26_17-11-39_384698_1636001/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-04-26_17-11-39_384698_1636001',
 'metrics_export_port': 59038,
 'gcs_address': '10.32.105.13:61182',
 'address': '10.32.105.13:61182',
 'node_id': '56df8b272e48470479d666ac672ee9f0a855673256d180916d5246f0'}

In [13]:
@ray.remote
def calc_ratio(data_all, eids_dict, record, eids_record, eids_nonrecord, endpoints):
    r_ds = []
    
    for endpoint in endpoints:
        eids_endpoint = eids_dict[endpoint]
        
        # record set
        eid_idxs_dict = {}
        eid_idxs_dict["record"] = np.where(np.in1d(eids_endpoint, eids_record, assume_unique=True))[0]
        eid_idxs_dict["nonrecord"] = np.where(np.in1d(eids_endpoint, eids_nonrecord, assume_unique=True))[0]

        for key, eid_idxs in eid_idxs_dict.items():
            eids_temp = eids_endpoint[eid_idxs]
            s = data_all[f"{endpoint}_event"].loc[eids_temp]
            n=s.sum()
            freq = n/len(s)
            
            if key=="record":
                s_record
                n_record = n
                freq_record = freq
                
            if key=="nonrecord":
                s_nonrecord = s
                n_nonrecord = n
                freq_nonrecord = freq
        
        jaccard = n_record / (n_nonrecord + len(eid_idxs_dict["record"])) 
        
        r_ds.append({"endpoint": endpoint, "n_eligable": len(eids_dict[endpoint]), 
                  "record": record, "n_records": len(eids_record), 
                  "n_events_record": n_record, "freq_events_record": freq_record,
                    "n_events_nonrecord": n_nonrecord, "freq_events_nonrecord": freq_nonrecord,
                    "jaccard": jaccard})
    return r_ds

In [14]:
d_nested = []
ref_data_all = ray.put(data_all)
ref_eids_dict = ray.put(eids_dict)
for record in tqdm(record_freqs.index):
    s_record = data_all[record]
    eids_record = s_record[s_record==True].index.values
    eids_nonrecord = s_record[s_record==False].index.values
    ref_results = calc_ratio.remote(ref_data_all, ref_eids_dict, record, eids_record, eids_nonrecord, endpoints)
    d_nested.append(ref_results)
d_nested = [ray.get(e) for e in tqdm(d_nested)]
del ref_data_all
del ref_eids_dict

  0%|          | 0/19366 [00:00<?, ?it/s]

(calc_ratio pid=1636273) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ratio pid=1636279) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ratio pid=1636271) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ratio pid=1636285) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ratio pid=1636284) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ratio pid=1636293) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ratio pid=1636283) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ratio pid=1636277) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars
(calc_ra

  0%|          | 0/19366 [00:00<?, ?it/s]

(calc_ratio pid=1636290) /tmp/ipykernel_1636001/3547803062.py:17: RuntimeWarning: invalid value encountered in long_scalars


In [15]:
from itertools import chain

d = list(chain(*d_nested))

In [16]:
endpoints_freqs = pd.DataFrame().from_dict(d)

In [17]:
endpoints_freqs.to_feather(f"{experiment_path}/record_inc_disease_freq_rnr.feather")

In [35]:
endpoints_ds = endpoints_freqs.query("n>100").sort_values("endpoint").reset_index(drop=True)

In [36]:
endpoints_ds.to_csv(f"{experiment_path}/endpoints.csv")

In [37]:
f"{experiment_path}/endpoints.csv"

'/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/endpoints.csv'